In [2]:
# The purpose of this file is to try regression algorithm for Diabetes Endotypes 
# created by Zhongyu Li, May 2024 
# Note: we will only use 6 cohort with HOMA2IR and HOMA2B data

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler


In [2]:
# first, we will load the data and conduct k means clustering to create the "TRUE labels"
# we will use the HOMA2IR and HOMA2B data to create the labels
# we will use the first 6 cohorts

path = '/Users/zhongyuli/Desktop/python/cluster analysis/dataset/data_array_homa2.csv'

data_6c = pd.read_csv(path) 

#drop missing values
data_6c = data_6c.dropna()
data_6c[:5]
#check the number of unique values in "study"
#data_6c['study'].value_counts()

,study_id,bmi,hba1c,ldlc,hdlc,tgl,sbp,dbp,ratio_th,dmagediag,glucosef2,insulinf2,study,homa2b,homa2ir
0,3278,33.87268,6.4,48.0,31.0,768,124.00000,78.00000,24.774193,56.01644,8.6025,1850.4,dppos,485.1,27.027027
1,8663,30.12909,9.6,97.0,36.0,197,121.66667,81.66667,5.472222,58.00000,9.5460,1806.0,mesa,427.8,27.027027
2,8543,22.68000,6.0,64.0,80.0,103,98.33333,46.33333,1.287500,77.00000,7.6590,1554.0,mesa,489.7,23.255814
4,2932,39.92760,6.1,100.0,45.0,252,113.00000,70.00000,5.600000,40.03288,10.1565,1238.4,dppos,297.6,20.833333
5,8535,39.66473,5.7,160.0,63.0,66,169.00000,81.00000,1.047619,88.00000,9.8235,1278.0,mesa,316.2,20.833333


In [5]:
# process data for k means 

study = data_6c['study']
study_id = data_6c['study_id']
data_6c_toscale = data_6c.drop(['study_id','study'], axis = 1)

scaler = StandardScaler() 
scaled_data = scaler.fit_transform(data_6c_toscale)
scaled_data[1:5]  

# Convert scaled data back to a DataFrame and add the study_id column back
scaled_data = pd.DataFrame(scaled_data, columns=data_6c_toscale.columns)
scaled_data[1:5]


,bmi,hba1c,ldlc,hdlc,tgl,sbp,dbp,ratio_th,dmagediag,glucosef2,insulinf2,homa2b,homa2ir
1,-0.411284,3.509045,-0.303468,-0.872041,0.596156,-0.156074,0.474103,0.690645,-0.387893,1.657598,10.578621,3.795082,8.958951
2,-1.458090,-0.326289,-1.260044,2.460115,-0.526872,-1.406717,-1.805932,-0.788316,1.074227,0.459786,8.942367,4.555503,7.531319
3,0.965681,-0.219752,-0.216506,-0.190463,1.253246,-0.620599,-0.278739,0.735804,-1.770529,2.045125,6.893154,2.195617,6.614265
4,0.928740,-0.645900,1.522723,1.172691,-0.968915,2.380946,0.431083,-0.873095,1.920717,1.833747,7.150280,2.424112,6.614265


In [6]:
# conduct k means clustering using FIVE variables method (method 3a and 3b)

kmeans = KMeans(
    init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57
)

# select five variables to cluster
selected_variables = ['bmi', 'hba1c', 'dmagediag','homa2b','homa2ir']
data_to_cluster = scaled_data[selected_variables]

kmeans = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
kmeans.fit(data_to_cluster)


# Add the labels to the scaled dataset
scaled_data['cluster'] = kmeans.labels_


In [18]:
scaled_data[1:5]
#summary of the cluster
scaled_data['cluster'].value_counts()
# summarize variables by cluster
pd.DataFrame(kmeans.cluster_centers_, columns=selected_variables)

,bmi,hba1c,dmagediag,homa2b,homa2ir
0,0.320450,3.335735,-0.480457,-0.808368,0.076310
1,0.656798,-0.052042,-0.859584,0.192885,0.235503
2,0.146482,0.041590,0.124030,2.399602,2.367685
3,-0.560046,-0.241022,0.692980,-0.425945,-0.527782


In [8]:
# add the labels to the original dataset but create a new dataset
data_6c['cluster'] = kmeans.labels_
data = data_6c.drop('study', axis=1)
data[1:5]

,study_id,bmi,hba1c,ldlc,hdlc,tgl,sbp,dbp,ratio_th,dmagediag,glucosef2,insulinf2,homa2b,homa2ir,cluster
1,8663,30.12909,9.6,97.0,36.0,197,121.66667,81.66667,5.472222,58.00000,9.5460,1806.0,427.8,27.027027,2
2,8543,22.68000,6.0,64.0,80.0,103,98.33333,46.33333,1.287500,77.00000,7.6590,1554.0,489.7,23.255814,2
4,2932,39.92760,6.1,100.0,45.0,252,113.00000,70.00000,5.600000,40.03288,10.1565,1238.4,297.6,20.833333,2
5,8535,39.66473,5.7,160.0,63.0,66,169.00000,81.00000,1.047619,88.00000,9.8235,1278.0,316.2,20.833333,2


In [9]:
# now run the multinomial logistic regression model
selected_variables2 = ['bmi', 'hba1c', 'ldlc', 'hdlc', 'tgl', 'sbp', 'dbp', 'ratio_th','dmagediag']

X = data[selected_variables2]  # Features
y = data.iloc[:, -1]   # Target labels (cluster from five variable method above) 
X[1:5]


,bmi,hba1c,ldlc,hdlc,tgl,sbp,dbp,ratio_th,dmagediag
1,30.12909,9.6,97.0,36.0,197,121.66667,81.66667,5.472222,58.00000
2,22.68000,6.0,64.0,80.0,103,98.33333,46.33333,1.287500,77.00000
4,39.92760,6.1,100.0,45.0,252,113.00000,70.00000,5.600000,40.03288
5,39.66473,5.7,160.0,63.0,66,169.00000,81.00000,1.047619,88.00000


In [10]:

# Define the features (X) and target labels (y)
X = scaled_data[selected_variables2]
y = scaled_data['cluster']

# Create the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Perform five-fold cross-validation and print the accuracy for each fold
scores = cross_val_score(model, X, y, cv=5)

print("Accuracy scores for each fold:")
print(scores)
print("Average accuracy across all folds:", np.mean(scores))


Accuracy scores for each fold:
[0.86811594 0.83309144 0.86502177 0.87953556 0.88098694]
Average accuracy across all folds: 0.8653503291895417


In [15]:
# Fit the logistic regression model
model.fit(X, y)


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [16]:
# Coefficients are stored in a (n_classes, n_features) array
coefficients = model.coef_


coef_df = pd.DataFrame(coefficients, columns=X.columns, index=[f'Class_{i}' for i in range(coefficients.shape[0])])
print("Coefficients for each cluster:")
print(coef_df)

#each class has its own set of coefficients, and these are interpreted relative to a hypothetical zero baseline (not another class). The coefficients for each class tell you the change in the log odds of that class versus not being in that class, given a one-unit change in the predictor.


Coefficients for each cluster:
              bmi     hba1c      ldlc      hdlc       tgl       sbp       dbp  \
Class_0  0.457699  3.472139 -0.153816 -0.062988 -0.148906  0.064711 -0.032404   
Class_1  1.002890 -1.122145  0.109349 -0.125754  0.310291  0.049401  0.101709   
Class_2  0.074766 -0.816276 -0.060829  0.204333 -0.436445 -0.000537 -0.234642   
Class_3 -1.535354 -1.533718  0.105296 -0.015592  0.275060 -0.113575  0.165338   

         ratio_th  dmagediag  
Class_0 -0.011643  -0.858852  
Class_1 -0.156432  -1.455903  
Class_2  0.635233   0.343463  
Class_3 -0.467157   1.971292  


In [13]:
from sklearn.metrics import classification_report, confusion_matrix

# Predictions to evaluate the model
y_pred = model.predict(X)
report = classification_report(y, y_pred)
conf_matrix = confusion_matrix(y, y_pred)

print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(conf_matrix)


Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       142
           1       0.87      0.93      0.90      1341
           2       0.07      0.00      0.01       245
           3       0.88      0.95      0.91      1718

    accuracy                           0.87      3446
   macro avg       0.69      0.71      0.69      3446
weighted avg       0.82      0.87      0.84      3446

Confusion Matrix:
[[ 135    5    0    2]
 [   1 1245    6   89]
 [   6  102    1  136]
 [   0   85    8 1625]]
